Now I have to generate the ground truth for my approach

In [1]:
### IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter


In [3]:
def get_image_relevant_data(df_info, df_positions,id_):
    if (id_ >= len(df_info)): 
        print("THIS ID IS NOT CONTAINED IN THE DATASET")
        return
    
    shape =[df_info[df_info['id'] == id_].reset_index()["height"][0],
            df_info[df_info['id'] == id_].reset_index()["width"][0]]
    kernel_number = df_info[df_info['id'] == id_].reset_index()["counts"][0]
    file_name = df_info[df_info['id'] == id_].reset_index()["file_name"][0]

    kernel_positions = df_positions[df_positions['image_id'] == id_][["kernel_x","kernel_y"]].to_numpy()    
    return kernel_positions, shape, kernel_number,file_name

In [4]:
def generate_label(label_info, image_shape,sig=1):
    """
    Generate a density map based on objects positions.
    Args:
        label_info: (x, y) objects positions
        image_shape: (width, height) of a density map to be generated
    Returns:
        A density map.
    """
    # create an empty density map
    label = np.zeros(image_shape, dtype=np.float32)

    # loop over objects positions and marked them with 100 on a label
    # note: *_ because some datasets contain more info except x, y coordinates
    for x, y, *_ in label_info:
        if y < image_shape[0] and x < image_shape[1]:
            label[int(y)][int(x)] = 100

    # apply a convolution with a Gaussian kernel
    label = gaussian_filter(label, sigma=(sig, sig), order=0)

    return label

In [13]:
root_path = "C:/Users/hilar/Documents/Data/GitHub/Vision_Cognitive_Systems_Proyect/Corn/corn_kenel_counting_dataset/corn_kenel_counting_dataset/"
sigma = 5

#for dataset in ["narrow","many","corn"]:
for dataset in ["narrow"]:
    #for mode in ["train","test"]:
    for mode in ["test"]:
        df_i = pd.read_csv (root_path + f"{dataset}_coco/{dataset}_kernel_{mode}_info.csv" )
        df_p = pd.read_csv (root_path + f"{dataset}_coco/{dataset}_kernel_{mode}_positions.csv" )       

        ### Loop over all the images in the dataset
        for im in range(len(df_i)):
            ### Get the information
            kernel_positions, shape, kernel_number, file = get_image_relevant_data(df_i, df_p, im)
            ### Produce the maps
            image = generate_label(kernel_positions, shape,sigma)
            ### Save map
            path_map = root_path + f"{dataset}_coco/" +file[:-4]+"_map"+file[-4:]
            plt.imsave(path_map, image)  
            
